In [20]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import select, update, delete

# Create an engine that connects to a SQLite database
engine = create_engine('sqlite:///:memory:', echo=True)

In [21]:
Base = declarative_base()

# Define a class that represents the users table
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    age = Column(Integer)

    def __repr__(self):
        return f"User(id={self.id}, name={self.name}, age={self.age})"

# Create the table in the database
Base.metadata.create_all(engine)

# Create a session object that manages the database transactions
Session = sessionmaker(bind=engine)
session = Session()

# Insert some data into the table using the User class
session.add_all([
    User(name='Alice', age=25),
    User(name='Bob', age=30),
    User(name='Charlie', age=35)
])
session.commit()

2023-05-17 00:08:00,010 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-17 00:08:00,012 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2023-05-17 00:08:00,014 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-17 00:08:00,015 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2023-05-17 00:08:00,016 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-17 00:08:00,019 INFO sqlalchemy.engine.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	age INTEGER, 
	PRIMARY KEY (id)
)


2023-05-17 00:08:00,020 INFO sqlalchemy.engine.Engine [no key 0.00138s] ()
2023-05-17 00:08:00,022 INFO sqlalchemy.engine.Engine COMMIT
2023-05-17 00:08:00,027 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-17 00:08:00,029 INFO sqlalchemy.engine.Engine INSERT INTO users (name, age) VALUES (?, ?)
2023-05-17 00:08:00,030 INFO sqlalchemy.engine.Engine [generated in 0.00114s] ('Alice', 25)
2023-05-17 00:08:00,032 INFO sqlalchemy.engine.Engine INSERT INTO users (n

# WHERE with ORM

In [30]:
stmt = select(User).where(User.age > 28)
for row in session.execute(stmt).fetchall():
    print(row[0])

2023-05-16 17:17:52,886 INFO sqlalchemy.engine.Engine SELECT users.id, users.name, users.age 
FROM users 
WHERE users.age > ?
2023-05-16 17:17:52,887 INFO sqlalchemy.engine.Engine [generated in 0.00151s] (28,)
<User(id=2, name=Bob, age=30)>
<User(id=3, name=Charlie, age=35)>
<User(id=5, name=Bob, age=30)>
<User(id=6, name=Charlie, age=35)>


In [25]:
print(session.execute(stmt).fetchall()[0][0])

2023-05-16 17:13:41,028 INFO sqlalchemy.engine.Engine SELECT users.id, users.name, users.age 
FROM users 
WHERE users.name = ?
2023-05-16 17:13:41,030 INFO sqlalchemy.engine.Engine [cached since 1460s ago] ('Alice',)
<User(id=1, name=Alice, age=25)>


In [14]:
u = User(name='dennis', age=58)
dir(u)

['__abstract__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__mapper__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__table__',
 '__tablename__',
 '__weakref__',
 '_sa_class_manager',
 '_sa_instance_state',
 '_sa_registry',
 'age',
 'id',
 'metadata',
 'name',
 'registry']

In [18]:
u

User(id=None, name=dennis, age=58)

In [16]:
u.__mapper__

TypeError: 'Mapper' object is not callable

In [32]:
session.add(
    User(name='dennis', age=58)
)
session.commit()

2023-05-16 17:22:34,018 INFO sqlalchemy.engine.Engine INSERT INTO users (name, age) VALUES (?, ?)
2023-05-16 17:22:34,020 INFO sqlalchemy.engine.Engine [cached since 1993s ago] ('dennis', 58)
2023-05-16 17:22:34,022 INFO sqlalchemy.engine.Engine INSERT INTO users (name, age) VALUES (?, ?)
2023-05-16 17:22:34,024 INFO sqlalchemy.engine.Engine [cached since 1993s ago] ('dennis', 58)
2023-05-16 17:22:34,025 INFO sqlalchemy.engine.Engine COMMIT


In [38]:
from sqlalchemy import update
stmt = (
    update(User)
    .where(User.name == "Alice")
    .values(name="Alice the Third von Baumgarten")
)
session.execute(stmt)
session.commit()

2023-05-16 17:29:32,567 INFO sqlalchemy.engine.Engine UPDATE users SET name=? WHERE users.name = ?
2023-05-16 17:29:32,569 INFO sqlalchemy.engine.Engine [cached since 22.71s ago] ('Alice the Third von Baumgarten', 'Alice')
2023-05-16 17:29:32,572 INFO sqlalchemy.engine.Engine COMMIT


In [7]:
session.execute(update(User), 
                [
                    User(id=1, name='Alice from Wonderland')
                ]
            )

ArgumentError: List argument must consist only of tuples or dictionaries

In [5]:
session.scalars(select(User).where(User.age > 28)).all()

2023-05-16 23:15:13,093 INFO sqlalchemy.engine.Engine SELECT users.id, users.name, users.age 
FROM users 
WHERE users.age > ?
2023-05-16 23:15:13,094 INFO sqlalchemy.engine.Engine [cached since 154.6s ago] (28,)


[User(id=2, name=Bob, age=30), User(id=3, name=Charlie, age=35)]

In [6]:
session.scalars(select(User)).all()


2023-05-16 23:16:14,308 INFO sqlalchemy.engine.Engine SELECT users.id, users.name, users.age 
FROM users
2023-05-16 23:16:14,310 INFO sqlalchemy.engine.Engine [generated in 0.00127s] ()


[User(id=1, name=Alice, age=25),
 User(id=2, name=Bob, age=30),
 User(id=3, name=Charlie, age=35)]

In [39]:
import pandas as pd
pd.read_sql('users', con=engine)

2023-05-16 17:29:34,510 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2023-05-16 17:29:34,510 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-16 17:29:34,514 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2023-05-16 17:29:34,515 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-16 17:29:34,516 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("users")
2023-05-16 17:29:34,517 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-16 17:29:34,519 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2023-05-16 17:29:34,519 INFO sqlalchemy.engine.Engine [raw sql] ('users',)
2023-05-16 17:29:34,521 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("users")
2023-05-16 17:29:34,522 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-16 17:29:34,524 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("users")
20

,id,name,age
0,1,Alice the Third von Baumgarten,25
1,2,Bob,30
2,3,Charlie,35
3,4,Alice the Third von Baumgarten,25
4,5,Bob,30
5,6,Charlie,35
6,7,dennis,58
7,8,dennis,58
